input data: country, HDI for year (индекс человеческого развития), gdp_for_year (ввп в год), gdp_per_capita (ввп на душу населения).

output data (m/f - пол группы, соответственно мужской и женский; два числа - это возрастная категория группы):

m_5-14, m_15-24, m_25-34, m_35-54, m_55-74, m_75-, f_5-14, f_15-24, f_25-34, f_35-54, f_55-74, f_75-.

method: простейшая нейронная сеть

data from
https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

In [126]:
%matplotlib inline  
import numpy as np
from numpy import array as na

import pandas as pd
import random
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import matplotlib
import matplotlib.pyplot as plt

In [165]:
a=pd.read_csv("data\suicide.csv")


#создаю массив входных столбцов для обучения, состав столбцов очевиден по названию, добавляю переменную gdp_for_year,
# чтобы нормально удалить повторяющиеся данные. В конце удаляю эти значения
X_hdi=a["HDI for year"]
X_gdp_p=a["gdp_per_capita ($)"]
X_pop=a[" gdp_for_year ($) "]

# для создания столбцов, отвечающих за страну надо переделать слово в число так, чтобы все числа были равнозначны, для этого
# использую OneHotEncoder, он каждой стране сопоставит вектор, причем все векторы будут равноудалены друг от друга.
c=a["country"]
label_encoder = LabelEncoder()
c = pd.Series(label_encoder.fit_transform(c))
c=c.values.reshape(-1, 1)

onehot_encoder = OneHotEncoder(sparse=False)
matrix = onehot_encoder.fit_transform(c)
X_c = pd.DataFrame(onehot_encoder.fit_transform(c)).T

# объединяю все входные данные в единый массив
X=np.vstack([X_c,X_hdi,X_gdp_p,X_pop]).T

#удаляю повторяющиеся данные и ненужный параметр gdp_for_year
X=pd.DataFrame(X).drop_duplicates().values
X=X[:,:-1]
X=X.astype('float')

Дальше я строю массив ответов. 

In [67]:
b=a.groupby(['country-year','sex','age'])['suicides/100k pop'].mean()
B=pd.DataFrame(b.reset_index(name = "count_num"))
table=pd.pivot_table(B, values=['count_num'], index=['sex','age'],columns=['country-year'] )

Z=list()
for i in B['country-year'].drop_duplicates():
    Z.append(table['count_num'][i].values)

In [189]:
#X

In [191]:
#Z

In [200]:
Z.astype('float')
1

1

Далее наконец происходит обучение, почти всё берётся из блокнота-примера.
Я не смогла понять, какого размера должен быть скрытый слой. Поэтому решила начать с одноуровневого персептрона.
Я так-же не разобралась с нейроном смещения, поэтому сделала так, как поняла

In [193]:
# Функции numpy и её операции покомпонентно работают с векторами

def σ(x):
    return np.exp(-x*x) / (1 + np.exp(-x))

def dσdx(x):
    return x * (1- x+x*x)

learning_throttle = 1.0
iterations = 10

In [194]:
np.random.seed(1)

#synapses
s_1 = np.random.random((12,103))   


Далее происходит ошибка. Если я правильно поняла, то дело в несовпадении размеров массивов.

In [196]:
for j in range(iterations):
    for x, z in zip(X, Z):
    
        # Применим нейронную сеть
        l_1 = x
        l_2 = σ(s_1 @ l_1)
        
        # Обратное распространение ошибки 
        ε_l_2 = z - l_2
        Δ_l_2 = ε_l_2 * dσdx(l_2) * learning_throttle
          
        # Коррекция весов
        s_1 += Δ_l_2 @ l_1.T
    
    if j % 10000 == 0:   # Only print the error every 10000 steps, to save time and limit the amount of output. 
        print("Error: " + str(np.max(np.abs(ε_l_2))))

    
print("Output after training")
print(l_2)    

ValueError: shapes (12,) and (103,) not aligned: 12 (dim 0) != 103 (dim 0)

Но думаю, что есть и другая ошибка. При просмотре значений Δ_l_2, он весь заполнен Nan, чего по-моему быть не должно. И на данный момент я не знаю, как это решить.

In [198]:
Δ_l_2

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan])